<a href="https://colab.research.google.com/github/almkrishna/Machine-Learning-based-Email-Classifier/blob/main/Machine_Learning_based_Email_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
  mails = pd.read_csv(filename, encoding='ISO-8859-1')
  print(f"File '{filename}' uploaded and loaded into DataFrame 'mails'.")


Saving sample_data.csv to sample_data.csv
File 'sample_data.csv' uploaded and loaded into DataFrame 'mails'.


In [3]:
mails.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [4]:
mails.tail()

,S. No.,Message_body,Label
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam
956,957,Just glad to be talking to you.,Non-Spam


In [5]:
#get rows and columns size
mails.shape

(957, 3)

In [6]:
# get null values
mails.isnull().sum()

,0
S. No.,0
Message_body,0
Label,0


In [7]:
#check if there are duplicates
mails.duplicated().sum()

np.int64(0)

In [8]:
#lets check the way the first 4 rows looks
for each in mails['Message_body'].head(4):
    print("\n\n")
    print(each)




Rofl. Its true to its name



The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free



Pity, * was in mood for that. So...any other suggestions?



Will ü b going to esplanade fr home?


In [9]:
#check if we have urls
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
urls = mails['Message_body'].str.extract(pattern)

In [10]:
# Check if there are any non-null values (i.e., extracted URLs)
if urls.notna().any().any():
    print("URLs were found in the 'Message_body' column.")
else:
    print("No URLs were found in the 'Message_body' column.")

URLs were found in the 'Message_body' column.


In [11]:
#removing emails keywords
#we will replace with ""
import re

pattern = r"e[-\s]?mail"
email_uniform = mails['Message_body'].str.replace(pattern , "email" , flags=re.I)

In [12]:
#working with numbers
pattern =r"([0-9]+)"
nums = mails["Message_body"].str.extractall(pattern)

In [13]:
import string
import re
x = string.punctuation
pattern =r"([0-9]+)"
def clearAll(Message_body):
    Message_body = Message_body.replace(string.punctuation , "")

    return Message_body

In [14]:
mails.head(4).apply(clearAll)

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam


In [15]:
# remove any non chars
for c in range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in string.punctuation]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [16]:
# pattern = r"([0-9]+)"
#remove the digits
number = ["1" , "2","3" ,'4' ,'5','6','7','8','9' ,"0"]
for c in  range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in number]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [17]:
#do vectorization and covert it to matrix
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

text = vec.fit_transform(mails['Message_body'])


In [18]:
#get the labels
y = mails['Label']

# Encode labels to numerical values as XGBoost expects [0, 1] for binary classification
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

text.shape

(957, 3022)

In [19]:
#do splitliting the data 80 to 20
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text , y , test_size = 0.2 , random_state = 0)

In [20]:
#training using naibayes classifier(multinomial)
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
#cross validation for naive bayes algo
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf1 , X = text, y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[1.         0.95833333 0.96875    1.         0.94791667 0.92708333
 0.94791667 0.98947368 0.96842105 0.93684211]
Mean for cross val is 0.9644736842105264
std for the cross val is 0.02435287022912144


In [22]:
#do training baesd on multinomialnb
clf1.fit(x_train, y_train)

MultinomialNB()

In [23]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred = clf1.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.99      1.00      1.00       673
           1       0.99      0.96      0.97        92

    accuracy                           0.99       765
   macro avg       0.99      0.98      0.98       765
weighted avg       0.99      0.99      0.99       765




Confudion Matrix  
 [[672   1]
 [  4  88]]



Accuracy score   0.9934640522875817


In [24]:
#checking score based on set test
pred = clf1.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.98      0.99      0.98       162
           1       0.93      0.87      0.90        30

    accuracy                           0.97       192
   macro avg       0.95      0.93      0.94       192
weighted avg       0.97      0.97      0.97       192




Confudion Matrix  
 [[160   2]
 [  4  26]]



Accuracy score   0.96875


In [25]:
#randomclassifier model
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier()


In [26]:
#cross validation for random classifeir
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf2 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.94791667 0.92708333 0.98958333 0.96875    0.9375     0.94791667
 0.92708333 0.95789474 0.94736842 0.93684211]
Mean for cross val is 0.948793859649123
std for the cross val is 0.018334435037828563


In [27]:
clf2.fit(x_train ,y_train)

RandomForestClassifier()

In [28]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred2 = clf2.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred2)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred2)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred2)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      1.00      1.00       673
           1       1.00      1.00      1.00        92

    accuracy                           1.00       765
   macro avg       1.00      1.00      1.00       765
weighted avg       1.00      1.00      1.00       765




Confudion Matrix  
 [[673   0]
 [  0  92]]



Accuracy score   1.0


In [29]:
#checking score based on set test
pred1 = clf2.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred1)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred1)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred1)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.91      1.00      0.95       162
           1       1.00      0.47      0.64        30

    accuracy                           0.92       192
   macro avg       0.96      0.73      0.79       192
weighted avg       0.92      0.92      0.90       192




Confudion Matrix  
 [[162   0]
 [ 16  14]]



Accuracy score   0.9166666666666666


In [30]:
# Function to preprocess a single message
def preprocess_message(message):
    # Remove punctuation
    message = ''.join([char for char in message if char not in string.punctuation])
    # Remove numbers
    message = ''.join([char for char in message if char not in number])
    # Convert to lowercase (important for consistent vectorization)
    message = message.lower()
    return message

# Define a list of new messages to test
new_messages_to_test = [
    "Congratulations! You've won a $1,000 gift card. Claim it now at our website: example.com/claim",
    "Hey, just wanted to check in and see how you're doing. Call me when you get a chance.",
    "URGENT! Your bank account has been compromised. Click this link immediately to secure your funds: tinyurl.com/secure",
    "Meeting at 3 PM today, don't forget your report.",
    "Your mobile number has won a £1000 prize. Please call 09061701461 from your landline to claim.",
    "Can we meet tomorrow at 10 AM to discuss the project update?"
]

print("--- Predicting for new messages ---")

for i, msg in enumerate(new_messages_to_test):
    print(f"\nMessage {i+1}: '{msg}'")
    # Preprocess the message
    cleaned_msg = preprocess_message(msg)
    print(f"Cleaned message: '{cleaned_msg}'")

    # Vectorize the cleaned message
    vectorized_msg = vec.transform([cleaned_msg])

    # Predict using the Multinomial Naive Bayes model
    prediction_label_encoded = clf1.predict(vectorized_msg)

    # Decode the prediction
    predicted_label = le.inverse_transform(prediction_label_encoded)

    print(f"Predicted as: {predicted_label[0]}")

--- Predicting for new messages ---

Message 1: 'Congratulations! You've won a $1,000 gift card. Claim it now at our website: example.com/claim'
Cleaned message: 'congratulations youve won a  gift card claim it now at our website examplecomclaim'
Predicted as: Spam

Message 2: 'Hey, just wanted to check in and see how you're doing. Call me when you get a chance.'
Cleaned message: 'hey just wanted to check in and see how youre doing call me when you get a chance'
Predicted as: Non-Spam

Message 3: 'URGENT! Your bank account has been compromised. Click this link immediately to secure your funds: tinyurl.com/secure'
Cleaned message: 'urgent your bank account has been compromised click this link immediately to secure your funds tinyurlcomsecure'
Predicted as: Spam

Message 4: 'Meeting at 3 PM today, don't forget your report.'
Cleaned message: 'meeting at  pm today dont forget your report'
Predicted as: Non-Spam

Message 5: 'Your mobile number has won a £1000 prize. Please call 09061701461 